In [1]:
!pip install flask-ngrok
!pip install tensorflow
!pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.6 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=9254f133e051ec164728276aed727e743e25abe5cefba542640cf4ef098d23c9
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask,request,render_template,jsonify
import pickle
import numpy as np

In [ ]:
!pip install pyngrok
!killall ngrok

ngrok: no process found


In [ ]:
from pyngrok import ngrok

    # Set your authtoken
ngrok.set_auth_token("2pcX2uAKByMvIeV1aSTz5oPhklX_6gzJpFWmghqFn9BVsJ95v")

    # Now connect
ngrok_tunnel = ngrok.connect(5000)
print(f"Public URL: {ngrok_tunnel.public_url}")

Public URL: https://3ce1-34-125-46-89.ngrok-free.app


In [ ]:
import base64
import os
import cv2
import numpy as np
from flask import Flask, render_template, request, jsonify
from tensorflow.keras.models import load_model
from werkzeug.utils import secure_filename
from deepface import DeepFace

app = Flask(__name__, template_folder="/content/drive/MyDrive/AS/templates")
print(f"Public URL: {ngrok_tunnel.public_url}")
UPLOAD_FOLDER = 'uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Load AI model for accident detection
accident_model = load_model("/content/drive/MyDrive/AS/best_accident_detection_model.h5")

# Allowed video file types
def allowed_file(filename):
    ALLOWED_EXTENSIONS = {'mp4', 'avi', 'mov', 'mkv'}
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

# Process video for accident detection
def process_video(filepath):
    cap = cv2.VideoCapture(filepath)
    frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_resized = cv2.resize(frame, (224, 224))
        frame_norm = frame_resized.astype("float32") / 255.0  # Normalize
        frames.append(frame_norm)

    cap.release()

    if not frames:
        return None

    frames_np = np.array(frames)
    num_frames = frames_np.shape[0]
    if num_frames < 16:
        return None  # Not enough frames to make a valid input
    elif num_frames % 16 != 0:
        pad_size = 16 - (num_frames % 16)
        frames_np = np.pad(frames_np, ((0, pad_size), (0, 0), (0, 0), (0, 0)), mode='constant')

    num_samples = frames_np.shape[0] // 16
    frames_np = frames_np[:num_samples * 16].reshape((num_samples, 16, 224, 224, 3))
    predictions = accident_model.predict(frames_np)
    avg_prediction = np.mean(predictions)
    return avg_prediction

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    if 'video' not in request.files:
        return "No video file provided", 400

    file = request.files['video']
    if file.filename == '':
        return "No selected file", 400

    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(filepath)
        prediction = process_video(filepath)
        os.remove(filepath)

        if prediction is None:
            return "Error processing video", 500

        threshold = 0.5
        result = "Accident Detected" if prediction > threshold else "No Accident Detected"
        return render_template("result.html", result=result)

    return "Invalid file type", 400

@app.route('/emotion', methods=['POST'])
def detect_emotion():
    try:
        # Ensure base64 is imported
        import base64

        # Get Base64 image data from request
        data = request.get_json()
        if "image" not in data:
            return jsonify({"error": "No image data provided"}), 400

        image_data = data["image"].split(",")[1]  # Extract Base64 content

        # Convert Base64 to OpenCV Image
        image_bytes = base64.b64decode(image_data)
        np_arr = np.frombuffer(image_bytes, np.uint8)
        frame = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)

        if frame is None:
            return jsonify({"error": "Invalid image format"}), 400

        # Save image temporarily for DeepFace
        temp_path = os.path.join(UPLOAD_FOLDER, "temp.jpg")
        cv2.imwrite(temp_path, frame)

        # Run DeepFace Analysis
        result = DeepFace.analyze(img_path=temp_path, actions=['emotion'])

        # Extract detected emotion
        emotion = result[0]['dominant_emotion']

        # Remove temp file
        os.remove(temp_path)

        return jsonify({"emotion": emotion})

    except Exception as e:
        return jsonify({"error": str(e)}), 500


if __name__ == "__main__":
    app.run()


Public URL: https://3ce1-34-125-46-89.ngrok-free.app


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
